In [ ]:
classes_leg = {
    1 : 'Sealed',
    2 : 'Trees',
    3 : 'Shrubs',
    4 : 'Herbaceous',
    5 : 'Wetlands',
    6 : 'Non_Vegetated',
    7 : 'Water',
    99 : 'Water_2'
}

xlsx = '/home/gisuser/autocls/sepinputs.xlsx'

out = '/mnt/g/autocls/exp1_sep'

import os
from glass.rd import tbl_to_obj
from glass.pys.oss import fprop
from glass.rst.lulc.sep import separability_matrix

df = tbl_to_obj(xlsx)

for i, row in df.iterrows():
    separability_matrix(
        row.filter1, row.imgs, classes_leg,
        os.path.join(out, f'{fprop(row.filter1, "fn")}.xlsx')
    )
    separability_matrix(
        row.rules1, row.imgs, classes_leg,
        os.path.join(out, f'{fprop(row.rules1, "fn")}.xlsx')
    )
    separability_matrix(
        row.rules2, row.imgs, classes_leg,
        os.path.join(out, f'{fprop(row.rules2, "fn")}.xlsx')
    )



In [ ]:
import numpy as np
import pandas as pd

from scipy.stats import multivariate_normal

import math as m

from osgeo import gdal, gdal_array

from glass.pys.oss import lst_ff
from glass.wt import obj_to_tbl

In [ ]:
def bhattacharyya_distance(mean1, cov1, mean2, cov2):
    dist = 0.125 * (mean2 - mean1).T @ np.linalg.inv(0.5 * (cov1 + cov2)) @ (mean2 - mean1)
    return dist

In [ ]:
featfiles = lst_ff(featfolder, file_format='.tif')

In [ ]:
# Open Data
refimg = gdal.Open(refrst, gdal.GA_ReadOnly)

feats = [gdal.Open(i, gdal.GA_ReadOnly) for i in featfiles]

In [ ]:
# Get Ref NoData Value

ndval = refimg.GetRasterBand(1).GetNoDataValue()

# Get band number for each raster
featbands = [i.RasterCount for i in feats]

# Get number of features
nfeat = sum(featbands)

In [ ]:
# Convert imgs to Array, remove nodata values and reshape
refnum = refimg.GetRasterBand(1).ReadAsArray()
refnum = refnum.reshape((-1, 1))

# Get Ref array without nodata value
refref = refnum[refnum != ndval]

In [ ]:
refref.shape

In [ ]:
# Get features array
featnum = np.zeros((
    refref.shape[0], nfeat),
    gdal_array.GDALTypeCodeToNumericTypeCode(
        feats[0].GetRasterBand(1).DataType
    )
)

f = 0
for r in range(len(feats)):
    for b in range(featbands[r]):
        a = feats[r].GetRasterBand(b+1).ReadAsArray()
        a = a.reshape((-1, 1))
        a = a[refnum != ndval]

        featnum[:, f] = a

        f += 1

In [ ]:
# ID classes
classes = np.unique(refref)

In [ ]:
# Create an array with the distribuitions
# for each class
clsdist = []
for cls in classes:
    clsdist.append(featnum[refref == cls])

In [ ]:
# Get Mean and Covariance matrix
means = []
covs = []

for sa in clsdist:
    means.append(np.mean(sa, axis=0))
    covs.append(np.cov(sa.T))

In [ ]:
# Compute Bhattacharyya distance
# for each pair of classes
ftbl_bd = []
ftbl_jf = []
for c in range(len(classes)):
    _bd = [classes_leg[classes[c]]]
    _jf = [classes_leg[classes[c]]]
    
    for _c in range(len(classes)):
        # Compute Bhattacharyya distance between two distributions
        bdistance = bhattacharyya_distance(
            means[c], covs[c],
            means[_c], covs[_c]
        )

        # Compute Jeffries-Matusita distance
        jeffries = 2 * (1 - m.exp(-bdistance))

        _bd.append(bdistance)
        _jf.append(jeffries)

    ftbl_bd.append(_bd)
    ftbl_jf.append(_jf)

In [ ]:
# Generate final table

cols = ['class'] + [classes_leg[c] for c in classes]
df_bd = pd.DataFrame(ftbl_bd, columns=cols)
df_jf = pd.DataFrame(ftbl_jf, columns=cols)

obj_to_tbl([df_bd, df_jf], out_tbl, sheetsName=[
    'Bhattacharyya', 'Jeffries'
])